In [4]:
import pandas as pd
import requests

In [5]:
with pd.HDFStore('./../data/bayarea_block_baseline.h5') as store:
    hh = store.households
    buildings = store.buildings
    jobs = store.jobs

In [6]:
base_url = 'http://localhost:5000/model_data/base'
geo = 'blockgroups'

In [7]:
data_dictionary = {}

In [8]:
buildings.head()

,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,nonres_rent_per_sqft,res_price_per_sqft,stories,year_built,redfin_sale_price,redfin_sale_year,redfin_home_type,costar_property_type,costar_rent,building_type_id,block_id
building_id,,,,,,,,,,,,,,,,,,,
1,742974,1,0.00,1,2029,2029.42425,0,2029.42425,0.0,302.769751,1,1945,NaN,NaN,,,,1,060411141003012
2,744961,1,0.00,1,2029,2029.42425,0,2029.42425,0.0,254.429279,1,1965,NaN,NaN,,,,1,060411311003059
3,1442641,1,53262.87,1,1568,1568.00000,0,1568.00000,0.0,183.474166,1,1964,NaN,NaN,,,,1,060855082023014
4,190969,2,245000.00,0,0,1266.00000,1595,1266.00000,0.0,0.000000,2,1992,340000.0,2003.0,Condo/Coop,,,3,060014308004003
5,308709,2,283500.00,0,0,1513.00000,1513,1513.00000,0.0,0.000000,1,1978,442000.0,2004.0,Condo/Coop,,,3,060014418002005


In [9]:
hh.income.describe()

count    2.608019e+06
mean     8.138398e+04
std      8.149481e+04
min     -2.000000e+04
25%      3.020000e+04
50%      6.090000e+04
75%      1.020000e+05
max      1.968504e+06
Name: income, dtype: float64

In [10]:
# Job Density

options = {'area_unit':'miles'}

data = pd.Series(requests.post(base_url + '/density/jobs/' + geo, options).json())
df = pd.DataFrame(data, columns=['job_density'])

data_dictionary['job_density'] = 'Job Density in Jobs per Square Mile'

In [11]:
# Population Density

options = {'area_unit':'miles'}

data = pd.Series(requests.post(base_url + '/density/population/' + geo, options).json())
df['pop_density'] = data

data_dictionary['pop_density'] = 'Population Density in Persons per Square Mile'

In [12]:
# Median Income

options = {'target_var':'income',
          'spatial_agg':'median'}

data = pd.Series(requests.post(base_url + '/descriptive/households/' + geo, options).json())
df['income_median'] = data

data_dictionary['income_median'] = 'Median Income'

In [13]:
# Mean Income

options = {'target_var':'income',
          'spatial_agg':'mean'}

data = pd.Series(requests.post(base_url + '/descriptive/households/' + geo, options).json())
df['income_mean'] = data

data_dictionary['income_mean'] = 'Mean Income'

In [14]:
# Number of Households

data = pd.Series(requests.get(base_url + '/descriptive/households/' + geo).json())
df['households'] = data

data_dictionary['households'] = 'Number of Households'

In [15]:
# Number of Persons

options = {'target_var':'persons',
          'spatial_agg':'sum'}

data = pd.Series(requests.post(base_url + '/descriptive/households/' + geo, options).json())
df['persons'] = data

data_dictionary['persons'] = 'Total Population'

In [16]:
# Number of Jobs

data = pd.Series(requests.post(base_url + '/descriptive/jobs/' + geo).json())
df['jobs'] = data

data_dictionary['Jobs'] = 'Number of Jobs'

In [17]:
# Households Renting
options = {'hownrent':'2'}
data = pd.Series(requests.post(base_url + '/descriptive/households/' + geo, options).json())
df['hh_rent'] = data

data_dictionary['hh_rent'] = 'Number of Households Renting their Homes'

In [18]:
# Households Own
options = {'hownrent':'1'}
data = pd.Series(requests.post(base_url + '/descriptive/households/' + geo, options).json())
df['hh_own'] = data

data_dictionary['hh_rent'] = 'Number of Households that Own their Homes'

In [19]:
# Number of Buildings
data = pd.Series(requests.get(base_url + '/descriptive/buildings/' + geo).json())
df['buildings'] = data

data_dictionary['buildings'] = 'Number of Buildings'

In [20]:
# Number of Residential Units

options = {'target_var':'residential_units',
          'spatial_agg':'sum'}

data = pd.Series(requests.post(base_url + '/descriptive/buildings/' + geo, options).json())
df['res_units'] = data

data_dictionary['res_units'] = 'Number of Residential Units'

In [21]:
# Residential Square Feet

options = {'target_var':'residential_sqft',
          'spatial_agg':'sum'}

data = pd.Series(requests.post(base_url + '/descriptive/buildings/' + geo, options).json())
df['res_sqft'] = data

data_dictionary['res_sqft'] = 'Total Residential Square Feet'

In [22]:
# Non-Residential Square Feet

options = {'target_var':'non_residential_sqft',
          'spatial_agg':'sum'}

data = pd.Series(requests.post(base_url + '/descriptive/buildings/' + geo, options).json())
df['non_res_sqft'] = data

data_dictionary['non_res_sqft'] = 'Total Non-Residential Square Feet'

In [23]:
df.head()

,job_density,pop_density,income_median,income_mean,households,persons,jobs,hh_rent,hh_own,buildings,res_units,res_sqft,non_res_sqft
06001400100,689.058298,1108.665072,114025.0,137803.985141,1346.0,2946.0,1831.0,212.0,1134.0,1497.0,1433.0,3756954.0,1533732.0
06001400200,22809.595446,8168.977419,74100.0,96481.841981,848.0,1877.0,5241.0,334.0,514.0,664.0,900.0,1446837.0,2683733.0
06001400300,3854.539340,11343.358630,54000.0,78521.399752,2419.0,4841.0,1645.0,1197.0,1222.0,1529.0,2595.0,2941465.0,779046.0
06001400400,4228.999517,14851.186292,54210.0,68762.547696,1866.0,4035.0,1149.0,1068.0,798.0,1094.0,1955.0,2323575.0,543824.0
06001400500,2946.215888,15757.190503,37730.0,56054.495226,1571.0,3578.0,669.0,976.0,595.0,924.0,1655.0,1769479.0,360549.0


In [24]:
naics = {11: 'Agriculture, Forestry, Fishing and Hunting',
         21: 'Mining, Quarrying, and Oil and Gas Extraction',
         22: 'Utilities',
         23: 'Construction',
         3133: 'Manufacturing',
         42: 'Wholesale Trade',
         4445: 'Retail Trade',
         4849: 'Transportation and Warehousing',
         51: 'Information',
         52: 'Finance and Insurance',
         53: 'Real Estate and Rental and Leasing',
         54: 'Professional, Scientific, and Technical Services',
         55: 'Management of Companies and Enterprises',
         56: 'Administrative and Support and Waste Management and Remediation Services',
         61: 'Educational Services',
         62: 'Health Care and Social Assistance',
         71: 'Arts, Entertainment, and Recreation',
         72: 'Accommodation and Food Services',
         81: 'Other Services (except Public Administration)',
         92: 'Public Administration'}

In [25]:
# Jobs by Sector

for key, value in naics.items():
    options = {'sector_id':str(key)}

    data = pd.Series(requests.post(base_url + '/descriptive/jobs/' + geo, options).json())
    df['jobs_' + str(key)] = data
    
    data_dictionary['jobs_' + str(key)] = 'Number of Jobs in ' + value

In [26]:
dd = pd.DataFrame(pd.Series(data_dictionary), columns=['Variable Definition'])
dd.index.name = 'Variable'

In [27]:
dd.to_csv('baseline_dictionary.csv')

In [28]:
df.to_csv('baseline_results_v2.csv')